In [9]:
import os
import pandas as pd
import re
import numpy as np

In [81]:
# tcai - all content has been merged to get_data.py

income_path = '../data/income/'
processed_path = '../data/processed/'

In [113]:
income = pd.read_csv(income_path + 'income.csv')
states_name = pd.read_csv(income_path + 'states_name.csv')
population = pd.read_csv(income_path + 'population.csv')
time_series_mobility = pd.read_csv(processed_path + 'time_series_mobility.csv')

In [114]:
income_split = np.split(income, income[income.isnull().all(1)].index)[1:]
income_clean = pd.DataFrame()
for state in income_split:
    state_name = state['County'][1:2].values
    state['state'] = state_name[0]
    if state_name == 'District of Columbia':
        income_clean = income_clean.append(state[1:2])
    else: 
        income_clean = income_clean.append(state[2:])

In [115]:
income_clean['state'] = income_clean['state'].str.lower()
income_clean['county'] = income_clean['County'].str.lower()
income_clean['income_2018'] = income_clean['2,018']
income_clean = income_clean.drop(['County','2,018'], axis=1)
income_clean = income_clean[['county','state','income_2018']]

In [116]:
income_clean['income_2018'] = income_clean['income_2018'].map(lambda x: int(re.sub(',','', x)))

In [117]:
population

,FIPS,State,Area_Name,POP_ESTIMATE_2018
0,0,US,United States,"327,167,434"
1,1000,AL,Alabama,"4,887,871"
2,1001,AL,Autauga County,"55,601"
3,1003,AL,Baldwin County,"218,022"
4,1005,AL,Barbour County,"24,881"
...,...,...,...,...
3189,56037,WY,Sweetwater County,"43,051"
3190,56039,WY,Teton County,"23,081"
3191,56041,WY,Uinta County,"20,299"
3192,56043,WY,Washakie County,"7,885"


In [118]:
states = population['State']

In [119]:
# find index of 
index_list = []
for state in states.unique():
    index_list.append(states[(states == state)].index[0])

In [120]:
population = population.drop(index_list)

In [121]:
population['Area_Name'] = population['Area_Name'].map(lambda x: re.sub(' County| City','', x)).str.lower()

In [122]:
dictionary = {short:long for (short,long) in zip(states_name['Code'], states_name['State'])}
population['State'] = population['State'].map(dictionary).str.lower()

In [124]:
population = population[['FIPS','State', 'Area_Name', 'POP_ESTIMATE_2018']]
population.columns = ['FIPS','state','county', 'pop_2018']

In [125]:
population['pop_2018'] = population['pop_2018'].map(lambda x: int(re.sub(',','', x)))

In [95]:
# merge income
income_combined = pd.merge(time_series_mobility, income_clean, on=['state','county'])

In [97]:
time_series_all  = pd.merge(income_combined, population, on=['state','county'], how='left')

In [109]:
population[(population['county'] == 'jefferson')]

,state,county,pop_2018
38,alabama,jefferson,659300
150,arkansas,jefferson,68114
281,colorado,jefferson,580233
362,florida,jefferson,14288
479,georgia,jefferson,15430
590,idaho,jefferson,29439
650,illinois,jefferson,37820
751,indiana,jefferson,32208
856,iowa,jefferson,18381
949,kansas,jefferson,18975


In [105]:
time_series_all[(time_series_all['pop_2018'].isna())]

,Unnamed: 0,Unnamed: 0.1,date,county,state,fips,cases,deaths,county_fips_all,timestamp,...,wetBulbAvg,wetBulbMax,retail_and_recreation_percent_change_from_baseline,residential_percent_change_from_baseline,transit_stations_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,workplaces_percent_change_from_baseline,parks_percent_change_from_baseline,income_2018,pop_2018
4271,498,498,2020-03-09,jefferson,louisiana,22051,1,0,22051,2020-03-09,...,62.1,64.9,NaN,NaN,NaN,NaN,NaN,NaN,51005,NaN
4272,606,606,2020-03-10,jefferson,louisiana,22051,1,0,22051,2020-03-10,...,65.4,68.8,NaN,NaN,NaN,NaN,NaN,NaN,51005,NaN
4273,745,745,2020-03-11,jefferson,louisiana,22051,1,0,22051,2020-03-11,...,65.6,70.1,NaN,NaN,NaN,NaN,NaN,NaN,51005,NaN
4274,917,917,2020-03-12,jefferson,louisiana,22051,1,0,22051,2020-03-12,...,66.6,70.4,NaN,NaN,NaN,NaN,NaN,NaN,51005,NaN
4275,1133,1133,2020-03-13,jefferson,louisiana,22051,3,0,22051,2020-03-13,...,68.6,71.8,NaN,NaN,NaN,NaN,NaN,NaN,51005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64900,52123,52123,2020-04-15,tensas,louisiana,22107,1,0,22107,2020-04-15,...,44.9,50.2,NaN,NaN,NaN,NaN,NaN,NaN,36630,NaN
64901,54533,54533,2020-04-16,tensas,louisiana,22107,1,0,22107,2020-04-16,...,49.0,57.6,NaN,NaN,NaN,NaN,NaN,NaN,36630,NaN
64902,56958,56958,2020-04-17,tensas,louisiana,22107,1,0,22107,2020-04-17,...,59.1,66.2,NaN,NaN,NaN,NaN,NaN,NaN,36630,NaN
64903,59392,59392,2020-04-18,tensas,louisiana,22107,1,0,22107,2020-04-18,...,58.6,65.6,NaN,NaN,NaN,NaN,NaN,NaN,36630,NaN


In [413]:
# time_series_all.to_csv(processed_path+'time_series_all.csv')